In [16]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
import tensorflow.keras.backend as kb
from backwardcompatibilityml import scores
from backwardcompatibilityml.tensorflow import helpers as tf_helpers
import copy

tf.enable_v2_behavior()
tf.random.set_seed(0)

In [17]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [18]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [19]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

In [20]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

model.fit(
    ds_train,
    epochs=3,
    validation_data=ds_test,
)

Epoch 1/3
469/469 [==============================] - 1s 2ms/step - loss: 0.3558 - accuracy: 0.9021 - val_loss: 0.1865 - val_accuracy: 0.9458
Epoch 2/3
469/469 [==============================] - 1s 1ms/step - loss: 0.1606 - accuracy: 0.9536 - val_loss: 0.1407 - val_accuracy: 0.9575
Epoch 3/3
469/469 [==============================] - 1s 1ms/step - loss: 0.1157 - accuracy: 0.9668 - val_loss: 0.1060 - val_accuracy: 0.9679


In [21]:
class BCCrossEntropyLoss(object):
    
    def __init__(self, h1, h2, lambda_c):
        self.h1 = h1
        self.h2 = h2
        self.lambda_c = lambda_c
        self.__name__ = "BCCrossEntropyLoss"
        self.cce_loss = tf.keras.losses.SparseCategoricalCrossentropy(
            reduction=tf.keras.losses.Reduction.SUM)
    
    def dissonance(self, h2_output, target_labels):
        cross_entropy_loss = self.cce_loss(target_labels, h2_output)
        return cross_entropy_loss
    
    def __call__(self, x, y):
        h1_output = tf.argmax(self.h1(x), axis=1)
        h2_output = self.h2(x)
        h1_diff = h1_output - y
        h1_correct = (h1_diff == 0)
        _, x_support = tf.dynamic_partition(x, tf.dtypes.cast(h1_correct, tf.int32), 2)
        _, y_support = tf.dynamic_partition(y, tf.dtypes.cast(h1_correct, tf.int32), 2)
        h2_support_output = self.h2(x_support)
        dissonance = self.dissonance(h2_support_output, y_support)
        new_error_loss = self.cce_loss(y, h2_output) + self.lambda_c * dissonance
        
        return tf.reduce_sum(new_error_loss)

In [22]:
lambda_c = 0.9
model.trainable = False

h2 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

bc_loss = BCCrossEntropyLoss(model, h2, lambda_c)

In [23]:
optimizer = tf.keras.optimizers.Adam(0.001)

In [24]:
tf_helpers.bc_fit(h2, training_set=ds_train, testing_set=ds_test, epochs=6, bc_loss=bc_loss, optimizer=optimizer)

Epoch 1/6
=============================================== Training loss: 25.0517
Epoch 2/6
=============================================== Training loss: 18.5583
Epoch 3/6
=============================================== Training loss: 26.4048
Epoch 4/6
=============================================== Training loss: 14.9898
Epoch 5/6
=============================================== Training loss: 7.7008
Epoch 6/6
=============================================== Training loss: 5.2657
Training done.


In [25]:
model.trainable = False
h2.trainable = False

In [26]:
h1_predicted_labels = []
h2_predicted_labels = []
ground_truth_labels = []
for x_batch_test, y_batch_test in ds_test:
    h1_batch_predictions = tf.argmax(model(x_batch_test), axis=1)
    h2_batch_predictions = tf.argmax(h2(x_batch_test), axis=1)
    h1_predicted_labels += h1_batch_predictions.numpy().tolist()
    h2_predicted_labels += h2_batch_predictions.numpy().tolist()
    ground_truth_labels += y_batch_test.numpy().tolist()

In [27]:
btc = scores.trust_compatibility_score(h1_predicted_labels, h2_predicted_labels, ground_truth_labels)
bec = scores.error_compatibility_score(h1_predicted_labels, h2_predicted_labels, ground_truth_labels)

print(f"lambda_c: {lambda_c}")
print(f"BTC: {btc}")
print(f"BEC: {bec}")

lambda_c: 0.9
BTC: 0.9932844302097324
BEC: 0.660436137071651
